In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/ANNDL/Challenge1

Mounted at /gdrive
/gdrive/My Drive/ANNDL/Challenge1


# Import libraries

In [ ]:
import random

seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

In [ ]:
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [ ]:
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from collections import Counter

import sklearn.model_selection as sklms
import sklearn.metrics as sklm

# Import Dataset

In [ ]:
# Loading the data and dividing between data and labels
data = np.load('public_data.npz', allow_pickle=True)

x_tv = data['data']
y_tv = data['labels']

# Removing the outliers
# Identify the first image of each outlier
shrek = x_tv[58]
troll = x_tv[412]

# Looking for all the outliers in the dataset
arrayShrek = []
arrayTroll = []
for i in range(len(x_tv)):
    if (x_tv[i] == shrek).all():
        arrayShrek = arrayShrek + [i]
    elif (x_tv[i] == troll).all():
        arrayTroll = arrayTroll + [i]

deleteArray = arrayShrek + arrayTroll

# Removing the outlier from the data and labels
x_tv = np.delete(x_tv, deleteArray, axis=0)
y_tv = np.delete(y_tv, deleteArray, axis=0)

# Removing the duplicates images
# Find all the duplicates
duplicates = []
for i in range(len(x_tv)):
    for j in range(i+1, len(x_tv)):
        if (x_tv[i] == x_tv[j]).all():
           duplicates = duplicates + [j]

# Delete the duplicates from the data and labels
x_tv = np.delete(x_tv, duplicates, axis=0)
y_tv = np.delete(y_tv, duplicates, axis=0)

In [ ]:
y_tv = np.array([0 if x == 'healthy' else 1 for x in y_tv])
# Compute the class weights to understand if oversampling has balanced the classes
itemCt = Counter(y_tv)
maxCt = float(max(itemCt.values()))
class_weights = {clsID : maxCt/numImg for clsID, numImg in itemCt.items()}

In [ ]:
# Splitting the dataset into training, validation and test
X_train_val, X_test, y_train_val, y_test = train_test_split(x_tv, y_tv, random_state=seed, test_size=200, stratify=y_tv)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=0.20, stratify = y_train_val)

In [ ]:
# Oversampling the dataset to reach an equal number of samples for each class
# Count the number of images that needs to be added to the unhealthy class
healthy = 0
for i in range(len(x_tv)):
    if y_tv[i] == 'healthy':
        healthy = healthy + 1

diff = healthy - (len(x_tv) - healthy)

In [ ]:
# Oversampling the dataset to reach an equal number of samples for each class
# Count the number of images that needs to be added to the unhealthy class
countOS = 0
newSamples=[]
number_of_os = np.count_nonzero(y_train == 0) - np.count_nonzero(y_train == 1)

for i in range(len(X_train)):
    if(y_train[i] == 1 and countOS < number_of_os):
        if(i%3 == 0):
            newEl = np.rot90(X_train[i])
        elif(i%3 == 1):
            newEl = np.fliplr(X_train[i])
        else:
            newEl = np.flipud(X_train[i])
        newSamples.append(newEl)
        countOS = countOS +1

# Add new samples
newSamples = np.array(newSamples)
X_train = np.concatenate((X_train, newSamples), axis=0)
ones = np.array([1 for i in range(number_of_os)])
y_train = np.concatenate((y_train, ones), axis=None)

# Convert labels to categorical
y_train = tfk.utils.to_categorical(y_train,2)
y_val = tfk.utils.to_categorical(y_val,2)
y_test = tfk.utils.to_categorical(y_test,2)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

print("input shape: ", input_shape)
print("output shape: ", output_shape)

input shape:  (96, 96, 3)
output shape:  (2,)


## Mixup

In [ ]:
# Mixup implementation from https://keras.io/examples/vision/mixup/
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 64
EPOCHS = 10

train_ds_one = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(BATCH_SIZE * 100)
    .batch(BATCH_SIZE)
)
train_ds_two = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(BATCH_SIZE * 100)
    .batch(BATCH_SIZE)
)
# Because we will be mixing up the images and their corresponding labels, we will be
# combining two shuffled datasets from the same training data.
train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))

In [ ]:
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


def mix_up(ds_one, ds_two, alpha=0.2):
    # Unpack two datasets
    images_one, labels_one = ds_one
    images_two, labels_two = ds_two
    batch_size = tf.shape(images_one)[0]

    # Sample lambda and reshape it to do the mixup
    l = sample_beta_distribution(batch_size, alpha, alpha)
    x_l = tf.reshape(l, (batch_size, 1, 1, 1))
    y_l = tf.reshape(l, (batch_size, 1))

    # Perform mixup on both images and labels by combining a pair of images/labels
    # (one from each dataset) into one image/label
    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return (images, labels)

In [ ]:
#this is the train set to be used with mixup, during training it should be used in place of x and y
train_ds_mu = train_ds.map(
    lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.2), num_parallel_calls=AUTO
)

# First (Simple) Model

In [ ]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):

    conv_1x1 = tfkl.Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)

    conv_3x3 = tfkl.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)
    conv_3x3 = tfkl.Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(conv_3x3)

    conv_5x5 = tfkl.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)
    conv_5x5 = tfkl.Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(conv_5x5)

    pool_proj = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = tfkl.Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed))(pool_proj)

    output = tfkl.Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5, pool_proj])

    return output

In [ ]:
def build_model_with_augmentation(l2_lambda, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)
    preprocessing = tf.keras.Sequential([
        tfkl.RandomContrast(0.5),
        tfkl.RandomTranslation(0.2,0.2),
        tfkl.RandomFlip("horizontal_and_vertical"),
        tfkl.RandomRotation(1),
        tfkl.RandomZoom(0.2),
    ], name='preprocessing')

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    preprocessing = preprocessing(input_layer)

    x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='conv0')(preprocessing)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)

    x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='conv1')(x)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)

    x = inception_module(x,
                      filters_1x1=64,
                      filters_3x3_reduce=96,
                      filters_3x3=128,
                      filters_5x5_reduce=16,
                      filters_5x5=32,
                      filters_pool_proj=32,
                      name='inception_1a')

    x = inception_module(x,
                      filters_1x1=128,
                      filters_3x3_reduce=128,
                      filters_3x3=192,
                      filters_5x5_reduce=32,
                      filters_5x5=96,
                      filters_pool_proj=64,
                      name='inception_1b')

    x = tfkl.MaxPooling2D(name='mp2')(x)

    y = x

    x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_2a')

    x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_2b')

    x = tfkl.Add()([x, y])

    y = x

    x = inception_module(x,
                      filters_1x1=192,
                      filters_3x3_reduce=96,
                      filters_3x3=208,
                      filters_5x5_reduce=16,
                      filters_5x5=48,
                      filters_pool_proj=64,
                      name='inception_3a')

    x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_3b')

    x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3c')


    x = tfkl.Add()([x, y])

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)

    x = tfkl.Dense(units=256, activation='relu',name='classification1', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda),
                   kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)
    x = tfkl.Dropout(0.5, seed=seed)(x)

    x = tfkl.Dense(units=128, activation='relu',name='classification2', kernel_regularizer=tf.keras.regularizers.l2(l2_lambda),
                   kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)
    x = tfkl.Dropout(0.5, seed=seed)(x)

    output_layer = tfkl.Dense(units=2, activation='softmax',name='Output')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNN')

    # Compile the model
    model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

    # Return the model
    return model

In [ ]:
model = build_model_with_augmentation()
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
lr_patience = 15
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True, mode='auto')

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
batch_size = 64
epochs = 256

# Train the model
history = model.fit(
    x = X_train/255,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val/255, y_val),
    callbacks = callbacks
).history

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model.save('SubmissionModelNoMaxPool')

In [ ]:
# model = tfk.models.load_model('SubmissionModel')

predicted_test = np.argmax(model.predict(X_test/255, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
# Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")

# Transfer Learning & Fine Tuning

## EfficientNet

### Transfer Learning

In [ ]:
lr_patience = 7
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='auto')

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
enet_model = tf.keras.applications.EfficientNetV2L(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96,96,3),
    pooling="avg",
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
)

tfk.utils.plot_model(convnet, show_shapes=True)

preprocessing = tf.keras.Sequential([
        tfkl.RandomTranslation(0.15,0.15, fill_mode="reflect", interpolation='bilinear'),
        tfkl.RandomFlip("horizontal_and_vertical"),
        tfkl.RandomRotation(0.15, interpolation='bilinear'),
        tfkl.RandomZoom(-0.2, 0.1, interpolation='bilinear')
], name='preprocessing')

enet_model.trainable = False

inputs = tfk.Input(shape=(96, 96, 3))

inputs = preprocessing(inputs)

x = enet_model(inputs)

outputs = tfkl.Dense(2, activation='softmax')(x)

enet_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

enet_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

enet_model.summary()

In [ ]:
tl_history = enet_model.fit(
    train_ds_mu,
    batch_size = 32,
    epochs = 512,
    validation_data = ((X_val), y_val),
    callbacks = callbacks,
).history

In [ ]:
enet_model.save('ENET')

predicted_test = np.argmax(enet_model.predict(X_test, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
 #Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")

### Fine Tuning

In [ ]:

enet_model.get_layer('efficientnetv2-l').trainable = True

N = 500
for i, layer in enumerate(enet_model.get_layer('efficientnetv2-l').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(enet_model.get_layer('efficientnetv2-l').layers):
   print(i, layer.name, layer.trainable)
enet_model.summary()

In [ ]:
lr_patience = 10
l2_lambda = 5e-5
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=lr_patience,
    factor=0.9,
    mode='min',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=40, restore_best_weights=True, mode='auto')

callbacks2 = [
    early_stopping,
    lr_scheduler
]

In [ ]:
enet_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

# Fine-tune the model
ft_history = enet_model.fit(
    x = tf.keras.applications.efficientnet_v2.preprocess_input(X_train),
    y = y_train,
    batch_size = 128,
    epochs = 512,
    validation_data = (tf.keras.applications.efficientnet_v2.preprocess_input(X_val), y_val),
    callbacks = callbacks2
).history

## ConvNeXt Base (Final Model)


### Transfer Learning

In [ ]:
# Declare callbacks
lr_patience = 7
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='auto', start_from_epoch=20)

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
convnet = tf.keras.applications.ConvNeXtBase(
    model_name="convnext_base",
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96,96,3),
    pooling="avg",
    classes=2,
    classifier_activation="softmax",
)
tfk.utils.plot_model(convnet, show_shapes=True)

# Preprocessing applied only during training
preprocessing = tf.keras.Sequential([
        tfkl.RandomTranslation(0.15,0.15, fill_mode="reflect", interpolation='bilinear'),
        tfkl.RandomFlip("horizontal_and_vertical"),
        tfkl.RandomRotation(0.15, interpolation='bilinear'),
        tfkl.RandomZoom(-0.2, 0.1, interpolation='bilinear')
], name='preprocessing')

# Freeze all layers
convnet.trainable = False

inputs = tfk.Input(shape=(96, 96, 3))

inputs = preprocessing(inputs)

x = convnet(inputs)

# Add just one dense layer
x = tfkl.Dense(units=64, activation='relu',
               kernel_initializer=tfk.initializers.HeUniform(seed=seed),
               name='classification2')(x)
# Add dropout to reduce overfitting
x = tfkl.Dropout(0.7, seed=seed)(x)

# Use softmax as output function
outputs = tfkl.Dense(2, activation='softmax')(x)

convnet = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile model using binary crossentropy because we have two classes
convnet.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

convnet.summary()

350926856/350926856 [==============================] - 17s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 convnext_base (Functional)  (None, 1024)              87566464  
                                                                 
 classification2 (Dense)     (None, 64)                65600     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 87632194 (334.29 MB)
Trainable params: 65730 (256.76 KB)
Non-trainable params: 87566464 (334.04 MB)
___________

In [ ]:
# Train with mixup data
tl_history = convnet.fit(
    train_ds_mu,
    batch_size = 32,
    epochs = 512,
    validation_data = ((X_val), y_val),
    callbacks = callbacks,
).history

Epoch 1/512
74/74 [==============================] - 67s 483ms/step - loss: 0.8062 - accuracy: 0.5360 - val_loss: 0.6562 - val_accuracy: 0.6817 - lr: 1.0000e-04
Epoch 2/512
74/74 [==============================] - 21s 279ms/step - loss: 0.6885 - accuracy: 0.5831 - val_loss: 0.6268 - val_accuracy: 0.7441 - lr: 1.0000e-04
Epoch 3/512
74/74 [==============================] - 21s 284ms/step - loss: 0.6634 - accuracy: 0.6251 - val_loss: 0.6161 - val_accuracy: 0.7495 - lr: 1.0000e-04
Epoch 4/512
74/74 [==============================] - 22s 302ms/step - loss: 0.6454 - accuracy: 0.6491 - val_loss: 0.5850 - val_accuracy: 0.7656 - lr: 1.0000e-04
Epoch 5/512
74/74 [==============================] - 23s 305ms/step - loss: 0.6256 - accuracy: 0.6753 - val_loss: 0.5550 - val_accuracy: 0.7946 - lr: 1.0000e-04
Epoch 6/512
74/74 [==============================] - 23s 304ms/step - loss: 0.6180 - accuracy: 0.6824 - val_loss: 0.5239 - val_accuracy: 0.8237 - lr: 1.0000e-04
Epoch 7/512
74/74 [===============

In [ ]:
convnet.save('SubmissionModel_ConvB_Final3_onlyTL-2')

In [ ]:
predicted_test = np.argmax(convnet.predict(X_test, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
# Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")

### Fine Tuning

In [ ]:
convnet = tfk.models.load_model('SubmissionModel_ConvB_Final3_onlyTL-2')

In [ ]:
# Set all layers as trainable
convnet.get_layer('convnext_base').trainable = True

N = 200
# Freeze first 200 layers
for i, layer in enumerate(convnet.get_layer('convnext_base').layers[:N]):
  layer.trainable=False

In [ ]:
# Declare callbacks
lr_patience = 5
l2_lambda = 1e-4
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=lr_patience,
    factor=0.9,
    mode='min',
    min_lr=1e-6
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, mode='auto', start_from_epoch=1)

callbacks2 = [
    early_stopping,
    lr_scheduler
]

In [ ]:
# Fine-tune the model with low learning rate
convnet.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-6), metrics='accuracy')

ft_history = convnet.fit(
    train_ds_mu,
    batch_size = 16,
    epochs = 256,
    validation_data = ((X_val), y_val),
    callbacks = callbacks2,
).history

Epoch 1/256
59/59 [==============================] - 84s 763ms/step - loss: 0.4461 - accuracy: 0.8185 - val_loss: 0.3286 - val_accuracy: 0.8710 - lr: 1.0000e-06
Epoch 2/256
59/59 [==============================] - 27s 465ms/step - loss: 0.4429 - accuracy: 0.8226 - val_loss: 0.3257 - val_accuracy: 0.8710 - lr: 1.0000e-06
Epoch 3/256
59/59 [==============================] - 26s 443ms/step - loss: 0.4575 - accuracy: 0.8258 - val_loss: 0.3236 - val_accuracy: 0.8667 - lr: 1.0000e-06
Epoch 4/256
59/59 [==============================] - 26s 446ms/step - loss: 0.4476 - accuracy: 0.8185 - val_loss: 0.3214 - val_accuracy: 0.8645 - lr: 1.0000e-06
Epoch 5/256
59/59 [==============================] - 26s 441ms/step - loss: 0.4429 - accuracy: 0.8226 - val_loss: 0.3200 - val_accuracy: 0.8656 - lr: 1.0000e-06
Epoch 6/256
59/59 [==============================] - 26s 448ms/step - loss: 0.4441 - accuracy: 0.8261 - val_loss: 0.3191 - val_accuracy: 0.8656 - lr: 1.0000e-06
Epoch 7/256
59/59 [===============

In [ ]:
predicted_test = np.argmax(convnet.predict(X_test, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
# Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")


Test set accuracy: 90.00%
Test set precision: 90.07%
Test set recall: 90.00%
Test set F1: 90.03%


In [ ]:
convnet.save('SubmissionModel_ConvB_Final4_FT_1')

## ConvNeXtLarge

### Transfer Learning

In [ ]:
lr_patience = 15
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='auto')

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
convnet = tf.keras.applications.ConvNeXtXLarge(
    model_name="convnext_xlarge",
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96,96,3),
    pooling="avg",
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
)

tfk.utils.plot_model(convnet, show_shapes=True)

convnet.trainable = False

l2_lambda = 2e-5

inputs = tfk.Input(shape=(96, 96, 3))

x = convnet(inputs)

x = tfkl.Dense(units=64, activation='leaky_relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification4')(x)
x = tfkl.Dropout(0.7, seed=seed)(x)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
convnet = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
convnet.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['accuracy'])

# Display model summary
convnet.summary()


In [ ]:
tl_history = convnet.fit(
    x = X_train,
    y = y_train,
    batch_size = 64,
    epochs = 256,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
convnet.save("CN_L_TF")
predicted_test = np.argmax(convnet.predict(X_test, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
# Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")

### Fine Tuning

In [ ]:

convnet = tfk.models.load_model('CN_L_TF')
convnet.get_layer('convnext_xlarge').trainable = True

# Freeze first N layers
N = 130
for i, layer in enumerate(convnet.get_layer('convnext_xlarge').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(convnet.get_layer('convnext_xlarge').layers):
   print(i, layer.name, layer.trainable)
convnet.summary()

lr_patience = 7
l2_lambda = 5e-5
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=lr_patience,
    factor=0.9,
    mode='min',
    min_lr=1e-8
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='auto')

callbacks2 = [
    early_stopping,
    lr_scheduler
]

convnet.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(5e-6), metrics='accuracy')

# Fine-tune the model
ft_history = convnet.fit(
    x = (X_train),
    y = y_train,
    batch_size = 32,
    epochs = 500,
    validation_data = (X_val, y_val),
    callbacks = callbacks2
).history

In [ ]:
convnet.save("CN_L_FT")
predicted_test = np.argmax(convnet.predict(X_test, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
# Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")

# Cross Validation

## EfficientNetV2L


### K-Fold

In [ ]:
lr_patience = 15
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True, mode='auto')

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
def build_model():
  enet_model = tf.keras.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96,96,3),
    pooling="avg",
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
  )

  preprocessing = tf.keras.Sequential([
          tfkl.RandomBrightness(0.2, value_range=(0,1)),
          tfkl.RandomTranslation(0.2,0.2),
          tfkl.RandomFlip("horizontal_and_vertical"),
          tfkl.RandomRotation(1),
          tfkl.RandomZoom(0.2),
      ], name='preprocessing')

  tfk.utils.plot_model(enet_model, show_shapes=True)

  enet_model.trainable = False

  l2_lambda = 2e-5

  inputs = tfk.Input(shape=(96, 96, 3))

  inputs = preprocessing(inputs)

  x = enet_model(inputs)

  x = tfkl.Dense(units=1024, activation='relu',kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification1')(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)

  x = tfkl.Dense(units=512, activation='relu',kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification2')(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)

  x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification3')(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)

  x = tfkl.Dense(units=128, activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification4')(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)

  outputs = tfkl.Dense(2, activation='softmax')(x)

  enet_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  enet_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

  # Return the model
  return enet_model

In [ ]:
# Define the number of folds for cross-validation
num_folds = 5
# Initialize lists to store training histories, scores, and best epochs
histories = []
scores = []
best_epochs = []

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train_val, y_train_val)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  # Build a new dropout model for each fold
  k_model = build_model()

  # Train the model on the training data for this fold
  history = k_model.fit(
    x = tf.keras.applications.efficientnet_v2.preprocess_input(X_train_val[train_idx]*255),
    y = y_train_val[train_idx],
    validation_data=(tf.keras.applications.efficientnet_v2.preprocess_input(X_train_val[valid_idx]*255), y_train_val[valid_idx]),
    batch_size = 256,
    epochs = 1000,
    callbacks = callbacks,
  ).history

  # Evaluate the model on the validation data for this fold
  score = k_model.evaluate(X_train_val[valid_idx]*255, y_train_val[valid_idx], verbose=0)
  scores.append(score[1])

  # Calculate the best epoch for early stopping
  best_epoch = len(history['loss']) - 50
  best_epochs.append(best_epoch)

  # Store the training history for this fold
  histories.append(history)

In [ ]:
# Calculate the average best epoch
avg_epochs = int(np.mean(best_epochs))
print(f"Best average epoch: {avg_epochs}")

In [ ]:
# Build the final model using the calculated average best epoch
final_model = build_model()

# Train the final model on the combined training and validation data
final_history = final_model.fit(
    x = tf.keras.applications.mobilenet_v3.preprocess_input(X_train_val*255),
    y = y_train_val,
    batch_size = 256,
    epochs = avg_epochs
).history

In [ ]:
model = final_model

final_model.save('SubmissionModel')

predicted_test = np.argmax(model.predict(X_test*255, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
 #Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")
enet_model = final_model

## MobileNetV3Small

### K-Fold

In [ ]:
lr_patience = 15
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=lr_patience,
    factor=0.9,
    mode='auto',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True, mode='auto')

callbacks = [
    early_stopping,
    lr_scheduler
]

In [ ]:
def build_model():

    enet_model = tf.keras.applications.MobileNetV3Small(
    input_shape=(96,96,3),
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling='avg',
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
    )

    preprocessing = tf.keras.Sequential([
            tfkl.RandomFlip("horizontal_and_vertical"),
            tfkl.RandomZoom(0.2),
        ], name='preprocessing')

    tfk.utils.plot_model(enet_model, show_shapes=True)

    enet_model.trainable = False

    l2_lambda = 2e-4

    inputs = tfk.Input(shape=(96, 96, 3))

    inputs = preprocessing(inputs)

    x = enet_model(inputs)

    x = tfkl.Dense(units=576, activation='relu',kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification2')(x)
    x = tfkl.Dropout(0.5, seed=seed)(x)

    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.HeUniform(seed=seed), kernel_regularizer=tf.keras.regularizers.l2(l2_lambda), name='classification3')(x)
    x = tfkl.Dropout(0.5, seed=seed)(x)

    # Add a Dense layer with 2 units and softmax activation as the classifier
    outputs = tfkl.Dense(2, activation='softmax')(x)

    # Create a Model connecting input and output
    enet_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
    enet_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate = 1e-4), metrics=['accuracy'])

    # Return the model
    return enet_model

In [ ]:
# Define the number of folds for cross-validation
num_folds = 10

# Initialize lists to store training histories, scores, and best epochs
histories = []
scores = []
best_epochs = []

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train_val, y_train_val)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  # Build a new dropout model for each fold
  k_model = build_model()

  # Train the model on the training data for this fold
  history = k_model.fit(
    x = tf.keras.applications.mobilenet_v3.preprocess_input(X_train_val[train_idx]*255),
    y = y_train_val[train_idx],
    validation_data=(tf.keras.applications.mobilenet_v3.preprocess_input (X_train_val[valid_idx]*255), y_train_val[valid_idx]),
    batch_size = 128,
    epochs = 1000,
    callbacks = callbacks,
  ).history

  # Evaluate the model on the validation data for this fold
  score = k_model.evaluate(X_train_val[valid_idx]*255, y_train_val[valid_idx], verbose=0)
  scores.append(score[1])

  # Calculate the best epoch for early stopping
  best_epoch = len(history['loss']) - 50
  best_epochs.append(best_epoch)

  # Store the training history for this fold
  histories.append(history)

In [ ]:
# Calculate the average best epoch
avg_epochs = int(np.mean(best_epochs))
print(f"Best average epoch: {avg_epochs}")

In [ ]:
# Build the final model using the calculated average best epoch
final_model = build_model()

# Train the final model on the combined training and validation data
final_history = final_model.fit(
    x = tf.keras.applications.mobilenet_v3.preprocess_input(X_train_val*255),
    y = y_train_val,
    batch_size = 256,
    epochs = avg_epochs
).history

In [ ]:
model = final_model

final_model.save('SubmissionModel')

predicted_test = np.argmax(model.predict(X_test*255, verbose="silent"), axis=-1)
ground_truth_test = np.argmax(y_test, axis=-1)
# Accuracy on test set.
accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")
 #Precision, recall, and F-score on test set.
precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")
enet_model = final_model

# Ensemble Test

In [ ]:
# Simple ensemble with majority voting
model1 = tf.keras.models.load_model('ConvNeXtBase_1')
model2 = tf.keras.models.load_model('ConvNeXtBase_2')
model3 = tf.keras.models.load_model('ConvNeXtXLarge')

predicted_test1= np.argmax(model1.predict(X_test, verbose="silent"), axis=-1)
predicted_test2 = np.argmax(model2.predict(X_test, verbose="silent"), axis=-1)
predicted_test3 = np.argmax(model3.predict(X_test, verbose="silent"), axis=-1)

predicted_test = predicted_test1 +predicted_test2 + predicted_test3

predicted_test = (predicted_test>= 2)
ground_truth_test = np.argmax(y_test, axis=-1)

accuracy_test = sklm.accuracy_score(ground_truth_test, predicted_test)
print(f"Test set accuracy: {accuracy_test:.2%}")

precision_test = sklm.precision_score(ground_truth_test, predicted_test, average="weighted")
recall_test = sklm.recall_score(ground_truth_test, predicted_test, average="weighted")
f_measure_test = sklm.f1_score(ground_truth_test, predicted_test, average="weighted")
print(f"Test set precision: {precision_test:.2%}")
print(f"Test set recall: {recall_test:.2%}")
print(f"Test set F1: {f_measure_test:.2%}")